In [1]:
import pandas as pd
# {"headers": {"Scores": ["id", "score"]}, "rows": {"Scores": [[1, 3.50], [2, 3.65], [3, 4.00], [4, 3.85], [5, 4.00], [6, 3.65]]}}
pd.DataFrame([[1, 3.50], [2, 3.65], [3, 4.00], [4, 3.85], [5, 4.00], [6, 3.65]], columns=["id", "score"]).to_csv("./Scores.txt", index=None)

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("178 rank scores").config("pyspark.sql.shuffle.partition", "4").getOrCreate()

Write an SQL query to rank the scores. The ranking should be calculated according to the following rules:

The scores should be ranked from the highest to the lowest.
If there is a tie between two scores, both should have the same ranking.
After a tie, the next ranking number should be the next consecutive integer value. In other words, there should be no holes between ranks.
Return the result table ordered by score in descending order.

The query result format is in the following example.

In [3]:
scores = spark.read.option("header", True).csv("./Scores.txt")
scores.show()

+---+-----+
| id|score|
+---+-----+
|  1|  3.5|
|  2| 3.65|
|  3|  4.0|
|  4| 3.85|
|  5|  4.0|
|  6| 3.65|
+---+-----+



In [5]:
from pyspark.sql.window import Window
from pyspark.sql.functions import dense_rank,col
Word_spec = Window.orderBy(col("score").desc())
scores = scores.withColumn("rank", dense_rank().over(Word_spec))
scores.show()

22/07/09 11:26:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+---+-----+----+
| id|score|rank|
+---+-----+----+
|  3|  4.0|   1|
|  5|  4.0|   1|
|  4| 3.85|   2|
|  2| 3.65|   3|
|  6| 3.65|   3|
|  1|  3.5|   4|
+---+-----+----+

